# Exploring Some Complex System Dynamics

We want to understand and draw actionable conclusions about some of these systems. Moreover, we want to do so 
* beforehand (i.e., knowledge about an impending financial crisis is better than observations while it's happening)
* in an actionable way (a plan to avoid or mitigate the crisis is more useful than a mere forecast of its occurrence)

In many cases, we observe phase-changes or other specific outcomes in similar systems, and we want to inspect the conditions under which we can avoid (or encourage) these outcomes in our own systems.

## Generative models and simulations

A key mechanism for investigating complex dynamics is a __generative model__.

Generative models are mathematical models, typically implemented via computation, which can yield information about some aspects of a system. So they are, in a way, machine learning models. But they are different from the most common, predictive ML models, that look at a set of outcomes (data) and try to predict future results (next quarter's sales) or more general results (transcribing speech).

> Generative models posit some __data generating process__ which may be similar to the dynamics in our systems.

The models -- also called simulations -- can then produce lots of outcomes based on some initial parameters and the data generating process. 

By looking at the process, the parameters, and the outcomes, we can learn about the behavior of such a system as parameters are changed.

We can then become aware of risks or opportunities that might otherwise be hidden.

## Some key systems models

Today, we'll look at a few classes of models that can reveal some of the challenging dynamics of complexity.

We'll take a look at ...
* network models
* automata

Next time we'll look at
* agent-based models
* path dependence

along with examples where they might apply.

## Network models

Many processes exhibit the consequences of network effects. These include both natural and social systems, including business systems.

We may want to exploit network effects to
* generate sales
* dominate/control a product category
* spread positive sentiment around our product or firm

Conversely we may want to avoid or disrupt network effects to
* lower systemic risks ("domino effect" cascading failures)
* protect intellectual property (limiting the spread of illicit use)
* protect secrets and first-mover advantage

But not every network exhibits the massive spread we may want to encourage or avoid.

__We can experiment on synthetic networks and learn critical dynamics__

### Erdos-Renyi Graphs

We can think of our scenario of interest as a graph:
* a set of nodes, which might represents individuals or firms,
* and a set edges connecting pairs of nodes, which might represent communications, transactions, or business relationships.

The Erdos-Renyi model (Paul Erdős, Alfréd Rényi, Edgar Gilbert, separate work ~1959) considers a family of graphs that contain some fixed number of nodes __n__ and some fixed probability __p__ that any two nodes are connected.

This is a simplistic model, but it produces interesting behavior.

If __p__ is close to zero, we can easily imagine that the graph is unlikely to be __connected__ or provide some route between all pairs of nodes.

On the other hand, if __p__ is close to one, it is not surprising that the graph ends up connected.

In between, things get interesting. Let's take a quick look at how the probability of an E-R graph being connected varies as __p__ changes.

How do we do this?

The easiest way to explore this -- and a mechanism I recommend because it works even when other methods are intractable or deeply complicated -- is to simulate the system and count outcomes.

In [ ]:
import networkx as nx
import numpy as np

def make_er_graph(n, p):
    G = nx.Graph()
    nodes = range(n)
    G.add_nodes_from(nodes)
    G.add_edges_from( (i, j) for i in nodes for j in nodes if  i > j and np.random.random() < p )
    return G

g = make_er_graph(20, 0.1)
nx.draw(g)

In [ ]:
nx.is_connected(g)

In [ ]:
g1 = make_er_graph(20, 0.5)
nx.draw(g1)

In [ ]:
nx.is_connected(g1)

In [ ]:
def test_connectivity_for_random_graph(n, p):
    return nx.is_connected(make_er_graph(n, p))

def prob_connected(n, p, samples):
    return sum( (test_connectivity_for_random_graph(n, p) for i in range(samples)) ) / samples

prob_connected(20, 0.2, 100)

In [ ]:
from matplotlib import pyplot as plt

n = 8
samples = 200
edge_probs = np.linspace(0, 1, 20)
connectivity_probs = [prob_connected(n, p, samples) for p in edge_probs]

plt.plot(edge_probs, connectivity_probs)

One interesting phenomenon is that this transition point becomes more sudden as the graph grows

In [ ]:
n = 20
connectivity_probs = [prob_connected(n, p, samples) for p in edge_probs]

plt.plot(edge_probs, connectivity_probs)

In [ ]:
n = 40
connectivity_probs = [prob_connected(n, p, samples) for p in edge_probs]

plt.plot(edge_probs, connectivity_probs)

Erdos and Renyi discovered that the critical value, at which the connectivity probability quickly transitions from 0 to 1, is $\frac{log(n)}{n}$

That number gets close to zero for large n.

In [ ]:
np.log(100)/100

### Takeaway

What are some takeaways from this experiment?

For large graphs with random connections, even a tiny probability of connection will likely connect the whole graph.

* This could be a good thing -- if you're spreading the word about your new product, or signing up folks to transact on your new money platform.

* But it could also be a terrible thing if the "message" being passed is a new virus or a pro-genocide meme.

In our experiment, we looked at the effect of connectivity. But we can just as easily fix a connectivity probability and ask about the effect of scaling the graph. 

Because $\frac{log(n)}{n}$ gets small as n gets big, we can say that for *any* connectivity probability (above zero), there will be a graph size large enough that it is ~100% likely to be connected.

In plain language: we've discovered the math behind the assertion that adding more people (or more anything) to a graph makes it inevitable that anything and everything can spread everywhere.

If this is the spread of ...
* a lifesaving product or knowledge, we have reason to rejoice
* our tech platform product, we had better be careful about unintended consequences of that product, because they can be everywhere
* financial instability due to propagated risk (as in 2008), we can see that failure was inevitable with scale

__How does this connect to the distributions we talked about earlier?__

Highly connected networks mean that when signals (info, memes, viruses, etc.) spread, the transmission is multiplicative (as we've seen with Covid and $R_0$) and, of course, the events are not independent -- they are linked by the relations in the graph. So spread through a network can lead to power-law distributions. Depending on the exponent, these may have fat tails and hide a large number of dramatic surprises that would not be expected from thin-tailed distributions.

A great overview of network effects in financial risk is https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3651864

#### More is Different

This phenomenon -- that "more is different" or quantity can become quality -- is the idea behind, and title of, a 1972 paper by Philip Anderson often regarded as inaugurating the study of complex systems: https://science.sciencemag.org/content/177/4047/393

Since humans and human institutions are used to linear change, this discovery of mechanisms behind sudden, non-linear change is a key tool in designing the outcomes we want in the world.

## Cellular automata

Another form of generative model, cellular automata are discrete systems with a small set of rules governing their evolution over (traditionally discrete) time.

CAs are interesting because even very simple ones can exhibit a range of behavior from highly regular to chaotic.

### One-dimensional (elementary) automata

We can imagine a 1-D CA state as a sequence of discrete "cells" each with its own state -- in the simplest case they are either active ("alive") or inactive ("dead")

In the subsequent timestep, a cell's state is determined by its previous state and its previous neighbors' states.

Since each cell can be either 0 or 1, there are 2^3 = 8 possible configurations for a cell and its neighbors:

`111, 110, 101, 100, 011, 010, 001, 000`

An automaton rule maps each of these configurations to a new state (either 0 or 1) for the center cell.

The binary representation of a rule is an 8-bit number, where each bit determines the new state of the center cell for one of these configurations. The rightmost bit corresponds to the first configuration (111), the next bit to the right corresponds to the second configuration (110), and so on.

So, the binary representation of a rule is simply a compact way to express how the rule maps each of the 8 possible configurations to a new state for the center cell. There are 256 possible elementary cellular automaton rules, each with a unique binary representation that can be converted to a decimal number for naming purposes (e.g., Rule 30, Rule 90).
For example, consider Rule 30. Its binary representation is 00011110. We can map the bits to the corresponding configurations like this:

```
111 110 101 100 011 010 001 000
0   0   0   1   1   1   1   0
```

According to this mapping, Rule 30 dictates that:

* If a cell and its neighbors are in configuration 111, the new state of the center cell will be 0.
* If a cell and its neighbors are in configuration 110, the new state of the center cell will be 0.
* If a cell and its neighbors are in configuration 101, the new state of the center cell will be 0.
* If a cell and its neighbors are in configuration 100, the new state of the center cell will be 1.
* If a cell and its neighbors are in configuration 011, the new state of the center cell will be 1.
* If a cell and its neighbors are in configuration 010, the new state of the center cell will be 1.
* If a cell and its neighbors are in configuration 001, the new state of the center cell will be 1.
* If a cell and its neighbors are in configuration 000, the new state of the center cell will be 0.

Three well-known elementary cellular automata have gained significant attention due to their distinct behaviors and properties:

* Rule 90 generates a fractal-like pattern known as the Sierpinski triangle. It exhibits less chaotic behavior compared to Rule 30, and its simple and symmetric patterns make it visually appealing. Rule 90 is represented by the binary number 01011010, which is 90 in decimal.

* Rule 30 is known for its chaotic and unpredictable behavior. It generates complex patterns from simple initial conditions, and parts of its output are even used as random number generators in some software. Rule 30 is represented by the binary number 00011110, which is 30 in decimal.

* Rule 110: Rule 110 is particularly interesting because it exhibits complex behavior and is known to be Turing complete, meaning it can simulate any Turing machine, given an appropriate initial condition. This property implies that Rule 110 can perform any computation, given enough time and space. Rule 110 is represented by the binary number 01101110, which is 110 in decimal.

These rules are fascinating because they demonstrate how simple rules can give rise to a wide range of behaviors, from simple and repetitive patterns to complex and unpredictable structures.

In [20]:
def rule90(left, center, right):
    return left ^ right

def generate_automata(size, steps, rule):
    initial_state = [0] * size
    initial_state[size // 2] = 1

    automata = [initial_state]

    for _ in range(steps - 1):
        current_state = automata[-1]
        next_state = [0] * size

        for i in range(size):
            left = current_state[(i - 1) % size]
            center = current_state[i]
            right = current_state[(i + 1) % size]

            next_state[i] = rule(left, center, right)

        automata.append(next_state)

    return automata

def print_automata(automata):
    for state in automata:
        print("".join("#" if cell else " " for cell in state))

size = 80
steps = 40

automata = generate_automata(size, steps, rule90)
print_automata(automata)

                                        #                                       
                                       # #                                      
                                      #   #                                     
                                     # # # #                                    
                                    #       #                                   
                                   # #     # #                                  
                                  #   #   #   #                                 
                                 # # # # # # # #                                
                                #               #                               
                               # #             # #                              
                              #   #           #   #                             
                             # # # #         # # # #                            
                            

In [21]:
def rule30(left, center, right):
    return left ^ (center or right)

automata = generate_automata(size, steps, rule30)
print_automata(automata)

                                        #                                       
                                       ###                                      
                                      ##  #                                     
                                     ## ####                                    
                                    ##  #   #                                   
                                   ## #### ###                                  
                                  ##  #    #  #                                 
                                 ## ####  ######                                
                                ##  #   ###     #                               
                               ## #### ##  #   ###                              
                              ##  #    # #### ##  #                             
                             ## ####  ## #    # ####                            
                            

In [23]:
def rule110(left, center, right):
    return (left and center and not right) or (left and not center and right) or (not left and center and right) or \
            (not left and center and not right) or (not left and not center and right)

automata = generate_automata(size, steps, rule110)


print_automata(automata)

                                        #                                       
                                       ##                                       
                                      ###                                       
                                     ## #                                       
                                    #####                                       
                                   ##   #                                       
                                  ###  ##                                       
                                 ## # ###                                       
                                ####### #                                       
                               ##     ###                                       
                              ###    ## #                                       
                             ## #   #####                                       
                            

### Optional exercise: Conway's Game of Life

Possibly the most famous CA of all time is the 2-D CA known as Conway's Game of Life.

Let's take a quick look: https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life

The Game of Life is fascinating for numerous reasons, but the one we're focusing on today is another key phenomenon of complex systems, __emergence__.

__Emergence__ is another way of describing the manifestation of interesting, potentially surprising, nonlinear outcomes from a simple set of mechanistic rules.

Game of Life has a trivial ruleset and a passive environment and gives rise to astonishing complexity.